In [ ]:
using Gridap, Gridap.Geometry, Gridap.Fields
using Gmsh, GridapGmsh
using GLMakie, DelimitedFiles, Interpolations
using LinearAlgebra, SparseArrays, KrylovKit
using ChainRulesCore, Zygote
using PartitionedArrays
using NLopt
using GridapMakie

import Gridap.CellData: Interpolable
import ChainRulesCore: rrule
import Gmsh: gmsh


main_path = "/Users/jayyao/Documents/Research/RamanInFluid/"
include(main_path*"Materials/Materials.jl")
include(main_path*"Module/Mesh_Periodic_Cir.jl")
include(main_path*"Module/Mesh_Periodic.jl")
include(main_path*"Module/Helper.jl")
include(main_path*"Module/GridapFE.jl")
include(main_path*"Module/Control.jl")
include(main_path*"Module/Model.jl")
include(main_path*"Module/Objective.jl")

In [ ]:
material = "Ag"
n_λ, k_λ = RefractiveIndex(material,main_path,true)
λ1 = 532
λ2 = 549
nf = sqrt(1.77)
nm1 = n_λ(λ1) + 1im * k_λ(λ1)
nm2 = n_λ(λ2) + 1im * k_λ(λ2)
μ = 1
R = 1e-10
g_flat_surface = 147
nm1,nm2

In [ ]:
hr = (λ1+λ2)/nf/4          # Height of Raman molecule
# Geometry parameters of the mesh
L = 200           # Length of the normal region
hair = 500 + hr       # Height of the air region
hs = 300 + hr         # Height of the source location in air
ht = 200 + hr         # Height of the target location in air
hd = 200          # Height of design domain
rd = 10
hsub = 100        # Height of substrate domain below design domain
dpml = 300        # Thickness of the PML
hrd = (hd, hr)
# Characteristic length (controls the resolution, smaller the finer)
# resol = 30        # Number of points per wavelength
l1 = 20      # Air
l2 = 1       # Design domain
l3 = l1           # PML

meshfile = "geometry.msh"
# geo_param = PeriodicGeometry(L, hair, hs, ht, hd, hsub, dpml, l1, l2, l3)
geo_param = PeriodicCirGeometry(L, hair, hs, ht, hd, rd, hsub, dpml, l1, l2, l3)
MeshGenerator(geo_param, meshfile)

LHp=(Inf, hair + hd)  # Start of PML for x,y > 0
LHn=(Inf, hsub)       # Start of PML for x,y < 0


ω1 = 2 * π / λ1
phys1 = PhysicalParameters(ω1, nf, nm1, nm1, μ, R, dpml, LHp, LHn, hd)
ω2 = 2 * π / λ2
phys2 = PhysicalParameters(ω2, nf, nm2, nm2, μ, R, dpml, LHp, LHn, hd)



In [ ]:
############  Optimization parameters #############
flag_f = true       # Turn on filter
flag_t = true       # Turn on threshold

# Filter and threshold paramters
r = (5, 5)  # Filter radius
β = 32.0                  # β∈[1,∞], threshold sharpness
η = 0.5                   # η∈[0,1], threshold center

α = 0.0 / (2 * 1000.0)    # Equivalent loss α = 1/2Q

# Number of subspace
K = 20

# Amplify g for NLopt
Amp = 1

# Sum over kx
nkx = 30
nparts = nkx / 2

Bp = false          # Matrix B depend on parameters?
pv = 1

# Foundary constraint parameters
c = (r[1])^4
lw = r[1]
ls = r[1]
ηe = fηe(lw / r[1])
ηd = fηd(lw / r[1])

control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bp, pv, c, ηe, ηd, hrd)

gridap = GridapFE(meshfile, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], ["Source"], flag_f, true)



In [ ]:
p_init = ones(gridap.np) * 0.5 #+ rand(gridap.np) * 1e-2
β_list = [8.0, 8.0, 16.0, 16.0, 32.0, 32.0, 32.0]

g_opt = 0
for bi = 1 : 6
    β = β_list[bi]
    if bi == 1
        g_opt, p_opt = g0_p_optimize(p_init, 1e-12, 70; phys1, phys2, control, gridap)
    
    else
        g_opt, p_opt = g0_p_optimize([], 1e-12, 70; phys1, phys2, control, gridap)
    end
    if isfile("p_opt.value.txt")
        run(`rm p_opt_value.txt`)
    end
    open("p_opt_value.txt", "w") do iop
        for i = 1 : length(p_opt)
            p_temp = p_opt[i]
            write(iop, "$p_temp \n")
        end
    end
    open("g_opt_value.txt", "a") do io
        write(io, "$g_opt \n")
    end
end
@show g_opt

In [ ]:
# Output and view
ImportFromFile = true
N = num_free_dofs(gridap.FE_U)
if ImportFromFile
    #p_max = readdlm("cloud/task5/p_opt_value.txt", Float64)
    p_max = readdlm("p_opt_value.txt", Float64)
    p0 = p_max[:]
#     p_cutoff = 0.5
#     p0[p0.<p_cutoff] .= 0.0
#     p0[p0.>p_cutoff] .= 1.0
else
    p0 = zeros(gridap.np)
end

kb = 0
usat = Inf
damp = 10

pf_vec = pf_p0(p0; control, gridap)
pfh = FEFunction(gridap.FE_Pf, pf_vec)
pth = (pf -> Threshold(pf; control)) ∘ pfh
A1_mat = MatrixA(pth, kb; phys=phys1, control, gridap)
b1_vec = assemble_vector(v->(∫(v)gridap.dΓ_s), gridap.FE_V)
u1_vec = A1_mat \ b1_vec
u1h = FEFunction(gridap.FE_U, u1_vec)

A2_mat = MatrixA(pth, kb; phys=phys2, control, gridap)
o_vec = VectorO(1, 1; gridap)
v2_vec = A2_mat' \ o_vec
v2h = FEFunction(gridap.FE_U, v2_vec)

@show g0_p(p0; kb=0, phys1, phys2, control, gridap, usat, damp)/L/g_flat_surface

fig, ax, plt = plot(gridap.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/(hd))
ax.title = "Design Shape"
limits!(ax, -L/2, L/2, 0, hd)
save("Results/shape.png", fig)
fig

In [ ]:
maxv = maximum(abs.(u1_vec))/1
maxe = 50#usat
e1=abs2(phys1.nf^2)
e2=abs2(phys1.nm^2)
#fig, ax, plt = plot(gridap.Ω, real(u1h), colormap = :bwr, colorrange=(-maxv, maxv))
#fig, ax, plt = plot(gridap.Ω, abs2(u1h), colormap = :hot, colorrange=(0, maxv^2))
fig, ax, plt = plot(gridap.Ω, 4*abs((conj(∇(u1h)) ⋅ ∇(u1h))/(e1 + (e2 - e1) * pth)), colormap = :hot, colorrange=(0, maxe))
Colorbar(fig[1,2], plt)
#ax.title = "|H|^2"
ax.title = "|E|^2"
#ax.aspect = AxisAspect(L/(phys1.LHp[2]+phys1.LHn[2]))
#limits!(ax, -L/2, L/2, -phys1.LHn[2], phys1.LHp[2])
ax.aspect = AxisAspect(L/(hd+hr*1))
limits!(ax, -L/2, L/2, 0, hd+hr*1)
save("Results/Field.png", fig)
fig

In [ ]:
p0 = rand(gridap.np)
δp = rand(gridap.np)*1e-8
grad = zeros(gridap.np)

kb = 0.0 * ω1
g0 = g0_p(p0, grad; kb, phys1, phys2, control, gridap)
g1 = g0_p(p0+δp, []; kb, phys1, phys2, control, gridap)
g1-g0, grad'*δp

In [ ]:
kb = 0
h_point = hd + 100
p0 = ones(gridap.np)
pf_vec = pf_p0(p0; control, gridap)
pfh = FEFunction(gridap.FE_Pf, pf_vec)
pth = (pf -> Threshold(pf; control)) ∘ pfh
A1_mat = MatrixA(pth, kb; phys=phys2, control, gridap)
b1_vec = assemble_vector(v->(∫(v*(x->GaussianD(x, [0, h_point], [2,2])))gridap.dΩ), gridap.FE_V)
u1_vec = A1_mat \ b1_vec
u1h = FEFunction(gridap.FE_U, u1_vec)


fig, ax, plt = plot(gridap.Ω, real(u1h), colormap = :bwr)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/(hd+hair))
ax.title = "Field"
limits!(ax, -L/2, L/2, 0, hd+hair)
#save("Results/shape.png", fig)
fig

In [ ]:
function p_bowtie(x, d, h, L, hd)
    dx = abs(x[1]) - d / 2 
    dy = abs(x[2] - hd / 2)
    bowtie = 0
    if dx >= 0 && dy <= h / (L - d) * dx
        bowtie = 1
    end
    return bowtie
end

function power_single_hr(hr)
    # Geometry parameters of the mesh
    L = 150           # Length of the normal region
    hair = 500 + hr       # Height of the air region
    hs = 300 + hr         # Height of the source location in air
    ht = 200 + hr         # Height of the target location in air
    hd = 100          # Height of design domain
    rd = 10           # Center radius
    hsub = 100        # Height of substrate domain below design domain
    dpml = 300        # Thickness of the PML
    hrd = [0, hd/2]
    # Characteristic length (controls the resolution, smaller the finer)
    resol = 30        # Number of points per wavelength
    l1 = L/resol      # Air
    l2 = l1/5.0       # Design domain
    l3 = l1           # PML

    meshfile = "geometry.msh"
    geo_param = PeriodicCirGeometry(L, hair, hs, ht, hd, rd, hsub, dpml, l1, l2, l3)
    MeshGenerator(geo_param, meshfile)
    
    ############  Optimization parameters #############
    flag_f = true       # Turn on filter
    flag_t = true       # Turn on threshold

    # Filter and threshold paramters
    r = [0.02 * L, 0.02 * L]  # Filter radius
    β = 80.0                  # β∈[1,∞], threshold sharpness
    η = 0.5                   # η∈[0,1], threshold center

    α = 0.0 / (2 * 1000.0)    # Equivalent loss α = 1/2Q

    # Number of subspace
    K = 20

    # Amplify g for NLopt
    Amp = 1

    # Sum over kx
    nkx = 30
    nparts = nkx / 2

    Bp = false          # Matrix B depend on parameters?
    pv = 1

    # Foundary constraint parameters
    c = 0#resol^4
    lw = r[1]
    ls = r[1]
    ηe = fηe(lw / r[1])
    ηd = fηd(lw / r[1])

    control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bp, pv, c, ηe, ηd, hrd)

    gridap = GridapFE(meshfile, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], ["Source"], flag_f)

    
    material = "Ag"
    n_λ, k_λ = RefractiveIndex(material,main_path,true)
    λ1 = 532
    λ2 = 549
    nm1 = n_λ(λ1) + 1im * k_λ(λ1)
    nm2 = n_λ(λ2) + 1im * k_λ(λ2)
    nf = sqrt(1.77)
    μ = 1
    R = 1e-10
    LHp=[Inf, hair + hd]  # Start of PML for x,y > 0
    LHn=[Inf, hsub]       # Start of PML for x,y < 0


    ω1 = 2 * π / λ1
    phys1 = PhysicalParameters(ω1, nf, nm1, nm1, μ, R, dpml, LHp, LHn, hd)
    ω2 = 2 * π / λ2
    phys2 = PhysicalParameters(ω2, nf, nm2, nm2, μ, R, dpml, LHp, LHn, hd)

    kb = 0
    h_point = hd + hr
    # p0 = rand(gridap.np)

    kb = 0
    d = 20
    h = 100
    bowtie_temp(v) = ∫(v * x->p_bowtie(x, d, h, L, hd))gridap.dΩ
    pc_vec = assemble_vector(bowtie_temp, gridap.FE_P)
    p_init = p_extract(pc_vec; gridap)
    p_init[p_init .< 0.1] .= 0
    p_init[p_init .> 0.1] .= 1
    
    pf_vec = pf_p0(p_init; control, gridap)
    pfh = FEFunction(gridap.FE_Pf, pf_vec)
    pth = (pf -> Threshold(pf; control)) ∘ pfh
    A1_mat = MatrixA(pth, kb; phys=phys2, control, gridap)
#     b1_vec = assemble_vector(v->(∫(v*(x->GaussianD(x, [0, h_point], [2,2])))gridap.dΩ), gridap.FE_V)
#     u1_vec = A1_mat \ b1_vec
    

#     O_mat = MatrixOl(phys2.ω, phys2.nf^2; gridap)
    O_vec = VectorO(1, 1; gridap)
    e_vec = A1_mat'\O_vec
    
    B1_mat = assemble_matrix(gridap.FE_U, gridap.FE_V) do u, v
        ∫((x->fr(x, λ2/nf/4, hd + hr - λ2/nf/4)) * (∇(v) ⋅ ∇(u)))gridap.dΩ
    end
    B2_mat = assemble_matrix(gridap.FE_U, gridap.FE_V) do u, v
        ∫((x->fr(x, λ2/nf/2, hd + hr - λ2/nf/2)) * (∇(v) ⋅ ∇(u)))gridap.dΩ
    end
    
    # return real(u1_vec'*O_mat*u1_vec), abs2(u1_vec'*O_vec)
    return real(e_vec'*B1_mat*e_vec), real(e_vec'*B2_mat*e_vec)
end

In [ ]:
Ng = 200
p_list = zeros(Ng)
pm_list = zeros(Ng)
h_list = zeros(Ng)

for hi = 1:Ng
    print(hi,"\n")
    λf = 549/sqrt(1.77)
    hr = (hi+40) * λf / 2 / 20
    h_list[hi] = hr/λf
    power1, power2 = power_single_hr(hr)
    p_list[hi] = power1
    pm_list[hi] = power2
    # pm_list[hi] = pm
end



In [ ]:
scene, layout = layoutscene(resolution=(1400,900))
ax = layout[1,1]=Axis(scene)
ms = 10
lin1 = lines!(ax,h_list,p_list)
# lin2 = lines!(ax,h_list,range(1,length(h_list),step=1).*g0_list)
lin2 = lines!(ax,h_list,pm_list)

#lin2 = scatter!(ax,range(1,12,step=1),g_r10,markersize=10)
#lin3 = lines!(ax,range(1,12,step=1),gr_r5.+1e-9)
#ax.yscale = log10
ax.xlabel="hr/λ"
ax.ylabel="Power"
#ax.title="Eigenvalue distribution"

# Legend(layout[1,1],[lin1, lin2],["Initial", "Optimized"])
# axislegend(ax, [lin1, lin2, lin3],["All","Far field","Difference"], position = :rt,
#     orientation = :vertical)
# save("Results/EigenValDistributionIO.png",scene)
scene